In [1]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
import torch.optim as optim
import os

class FacialModel(nn.Module):
    def __init__(self):
        super(FacialModel, self).__init__()
        self.fc1 = nn.Linear(3 * 224 * 224, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 17)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


data_dir = 'Celebrity Faces Dataset'
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])


train_dataset = ImageFolder(data_dir, transform=transform)
trainloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = FacialModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


writer = SummaryWriter()

for images, _ in trainloader:
    images = images.to(device)
    break

writer.add_graph(model, images)

for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 100 == 99:
            writer.add_scalar('training loss', running_loss / 100, epoch * len(trainloader) + i)
            running_loss = 0.0

for name, param in model.named_parameters():
    writer.add_histogram(name, param, global_step=epoch)

print('Finished Training')

writer.close()



Finished Training


In [2]:
%tensorboard --logdir runs

UsageError: Line magic function `%tensorboard` not found.
